In [1]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence)== 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f316d926a90>


In [6]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [10]:
#1개의 임베딩 2개 LSTM 1개 덴스

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)
#70001은 덴스 출력 단어읙 ㅐ수 256은 임베당 = 배치 사이즈 LSTM 데이터셋의 max_len이 20이다.

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-1.0429178e-05, -2.2635027e-04,  4.5041252e-06, ...,
         -2.1242040e-04,  9.0703747e-05, -1.0811833e-04],
        [-1.7545465e-04, -4.3935591e-04, -2.4784234e-04, ...,
         -4.5276713e-04,  1.5890462e-04, -3.4225409e-04],
        [-5.7580939e-04, -2.2662655e-04, -7.1539072e-04, ...,
         -8.2272361e-04, -5.5937766e-05, -1.9885671e-04],
        ...,
        [-2.7187623e-04,  4.3934156e-04,  3.6222808e-04, ...,
         -1.3715411e-03,  1.2900613e-03,  1.3839084e-03],
        [-6.3946654e-06, -1.7743021e-05,  4.0946971e-04, ...,
         -1.7753521e-03,  1.3875613e-03,  1.3991606e-03],
        [ 2.5430863e-04, -4.5857730e-04,  4.7478761e-04, ...,
         -2.1982149e-03,  1.5429682e-03,  1.4349202e-03]],

       [[-1.0429178e-05, -2.2635027e-04,  4.5041252e-06, ...,
         -2.1242040e-04,  9.0703747e-05, -1.0811833e-04],
        [ 2.7476011e-05, -4.8110509e-04,  1.9153670e-04, ...,
         -8.4879249e-04,  

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=1)

93/93 [==============================] - 34s 335ms/step - loss: 4.3734


In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he , , , , , , , , , , , , <end> '

In [16]:
import glob
import os
from sklearn.model_selection import train_test_split

#================================================================

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


In [29]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다 15개 이상은 건너뜀
    if len(sentence) == 0: continue
    if len(sentence) >= 20: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:30]

['<start> hook <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> skrrt intro kyla <end>',
 '<start> grips on your waist <end>',
 '<start> front way , back way <end>',
 '<start> streets not safe <end>',
 '<start> even when i m away <end>',
 '<start> i need a one dance <end>',
 '<start> i need a one dance <end>',
 '<start> tell me <end>',
 '<start> where , where , where <end>',
 '<start> where , where , where <end>',
 '<start> i need a one dance <end>',
 '<start> i need a one dance <end>',
 '<start> why you never alone <end>',
 '<start> you was in the zone <end>',
 '<start> we know ,

In [18]:
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  565    3 ...    0    0    0]
 [   2   67  119 ...    0    0    0]
 [   2   67  119 ...    0    0    0]
 ...
 [   2  328    3 ...    0    0    0]
 [   2  454   11 ...    0    0    0]
 [   2   11 2286 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f316da63bd0>


In [19]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx > 20: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : you
7 : .
8 : the
9 : it
10 : me
11 : a
12 : oh
13 : and
14 : to
15 : s
16 : my
17 : t
18 : yeah
19 : !
20 : in
21 : on


In [20]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 565   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[565   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [21]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset


#데이터셋 분리


enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=16)

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))

dataset = dataset.shuffle(BUFFER_SIZE)

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((256, 18), (256, 18)), types: (tf.int32, tf.int32)>

In [22]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


Source Train: (20491, 18)
Target Train: (20491, 18)


In [23]:
embedding_size = 20
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [24]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 18, 12001), dtype=float32, numpy=
array([[[-2.49370769e-05, -2.57131287e-05, -1.51605436e-05, ...,
          3.97117146e-05, -5.19081623e-05, -3.89201123e-05],
        [-8.83231769e-06, -1.44834939e-05, -1.01901365e-04, ...,
          2.94385645e-05, -1.06332744e-04, -3.18659249e-06],
        [ 3.92802649e-05,  1.10077526e-04, -1.79695460e-04, ...,
          1.87213857e-06, -1.57610746e-04,  8.76423437e-05],
        ...,
        [ 5.94694167e-04, -6.01621461e-04, -3.59018566e-04, ...,
         -6.86002139e-04,  1.89750805e-03,  1.51368818e-04],
        [ 6.38517377e-04, -6.54910051e-04, -3.93135502e-04, ...,
         -7.15013477e-04,  2.00042687e-03,  1.73410837e-04],
        [ 6.75473304e-04, -6.99368131e-04, -4.23403864e-04, ...,
         -7.35630165e-04,  2.08742055e-03,  1.91937186e-04]],

       [[-2.49370769e-05, -2.57131287e-05, -1.51605436e-05, ...,
          3.97117146e-05, -5.19081623e-05, -3.89201123e-05],
        [-9.37475343e-05, -1.01676596e-04,  5

In [25]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  240020    
_________________________________________________________________
lstm_2 (LSTM)                multiple                  4280320   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 25,214,069
Trainable params: 25,214,069
Non-trainable params: 0
_________________________________________________________________


In [26]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)



In [27]:
model.fit(enc_train, dec_train, epochs=10,batch_size=256, validation_data=(enc_val, dec_val),verbose=1)


Epoch 1/10
81/81 [==============================] - 34s 386ms/step - loss: 3.5269 - val_loss: 1.5866
Epoch 2/10
81/81 [==============================] - 30s 374ms/step - loss: 1.5029 - val_loss: 1.4143
Epoch 3/10
81/81 [==============================] - 30s 375ms/step - loss: 1.3109 - val_loss: 1.2686
Epoch 4/10
81/81 [==============================] - 30s 375ms/step - loss: 1.2242 - val_loss: 1.2428
Epoch 5/10
81/81 [==============================] - 30s 375ms/step - loss: 1.1902 - val_loss: 1.2272
Epoch 6/10
81/81 [==============================] - 30s 375ms/step - loss: 1.1638 - val_loss: 1.2163
Epoch 7/10
81/81 [==============================] - 30s 374ms/step - loss: 1.1529 - val_loss: 1.2060
Epoch 8/10
81/81 [==============================] - 30s 375ms/step - loss: 1.1307 - val_loss: 1.1980
Epoch 9/10
81/81 [==============================] - 30s 375ms/step - loss: 1.1442 - val_loss: 1.2263
Epoch 10/10
81/81 [==============================] - 30s 376ms/step - loss: 1.1329 - val_lo

In [28]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it <end> '